In [1]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [2]:
thisDir = dirname(@__FILE__())
any(path -> path == thisDir, LOAD_PATH) || push!(LOAD_PATH, thisDir)

3-element Array{Any,1}:
 "/home/stefan/looseends/julia-903644385b/local/share/julia/site/v0.6"
 "/home/stefan/looseends/julia-903644385b/share/julia/site/v0.6"      
 ""                                                                   

In [3]:
using CliffordOperators: PauliOperator, d_bellstatestabs, expand_2toN, CliffordOperator, clifford_group, ⊗

In [4]:
@everywhere using CliffordOperators: PauliOperator, d_bellstatestabs, expand_2toN, CliffordOperator, clifford_group, ⊗

In [5]:
using IterTools

bellpair_basis = collect(values(d_bellstatestabs))
pair1 = [[expand_2toN(4,1,3,op) for op in s] for s in bellpair_basis]
pair2 = [[expand_2toN(4,2,4,op) for op in s] for s in bellpair_basis]
twopairs_basis_set = Set([Set([p1 ; p2]) for (p1, p2) in product(pair1,pair2)]);

C = collect(clifford_group(2));

In [11]:
@everywhere function preserves(op::CliffordOperator)
    global twopairs_basis_set
    (map(s->map(p->op*p, s),
         twopairs_basis_set)
     ==twopairs_basis_set)
end

In [6]:
@everywhere preserves(op::CliffordOperator) = (
    global twopairs_basis_set;
    map(s->map(p->op*p, s),
        twopairs_basis_set)
    ==twopairs_basis_set)

In [7]:
blah = CliffordOperator([PauliOperator("XXXX") for i in 1:4],[PauliOperator("ZXXX") for i in 1:4])

+ZIII → +XXXX
+IZII → +XXXX
+IIZI → +XXXX
+IIIZ → +XXXX
+XIII → +ZXXX
+IXII → +ZXXX
+IIXI → +ZXXX
+IIIX → +ZXXX


In [9]:
whos()

                          Base               Module
                             C   1352 KB     11520-element Array{CliffordOperat…
             CliffordOperators  20978 KB     Module
                 Combinatorics  21073 KB     Module
                        Compat  21022 KB     Module
                          Core               Module
                        IJulia  21269 KB     Module
                     IterTools  21050 KB     Module
                          JSON  21089 KB     Module
                          Main               Module
                       MbedTLS  21115 KB     Module
                   Polynomials     76 KB     Module
                           ZMQ  21069 KB     Module
                bellpair_basis    352 bytes  4-element Array{Tuple{CliffordOper…
                          blah    464 bytes  CliffordOperators.CliffordOperator
                         pair1    480 bytes  4-element Array{Array{CliffordOper…
                         pair2    480 bytes  4-elemen

In [13]:
@spawnat 2 whos()

Future(2, 1, 17, Nullable{Any}())

	From worker 2:	                          Base               Module
	From worker 2:	             CliffordOperators  18711 KB     Module
	From worker 2:	                 Combinatorics    107 KB     Module
	From worker 2:	                        Compat     45 KB     Module
	From worker 2:	                          Core               Module
	From worker 2:	                     IterTools     55 KB     Module
	From worker 2:	                          Main               Module
	From worker 2:	                   Polynomials     76 KB     Module
	From worker 2:	                          blah    464 bytes  CliffordOperators.CliffordOperator
	From worker 2:	                     preserves      0 bytes  #preserves


In [12]:
fetch(@spawn preserves(blah))

LoadError: [91mOn worker 3:
[91mUndefVarError: twopairs_basis_set not defined[39m
preserves at ./In[11]:3
#41 at ./distributed/macros.jl:20
#103 at ./distributed/process_messages.jl:264 [inlined]
run_work_thunk at ./distributed/process_messages.jl:56
run_work_thunk at ./distributed/process_messages.jl:65 [inlined]
#96 at ./event.jl:73[39m

In [16]:
inds = collect(1:length(C))
len = length(inds)
bell_diagonal_op = SharedArray{Int,1}((len^2,));

In [27]:
using ProgressMeter

In [28]:
@showprogress 1 for i in inds
    @sync @parallel for j in inds
        op = C[i] ⊗ C[j]
        if preserves(op)
            bell_diagonal_op[i*len+j]=1
        end
        mylast = i,j
    end
end

Progress:   0%  ETA: 6:18:01

LoadError: [91mInterruptException:[39m

In [75]:
length(bell_diagonal_op)

128

In [76]:
11520/128

90.0

In [24]:
#f = open("julia-subgroup.jls", "w")
#serialize(f, bell_diagonal_op)
#close(f)

In [72]:
f = open("julia-subgroup.jls", "r")
bell_diagonal_op_ = deserialize(f)
close(f)

In [74]:
bell_diagonal_op

128-element Array{CliffordOperator,1}:
 +ZIII → +ZIII
+IZII → +IZII
+IIZI → +IIZI
+IIIZ → +IIIZ
+XIII → +XIII
+IXII → +IXII
+IIXI → +IIXI
+IIIX → +IIIX

 +ZIII → +IZII
+IZII → +ZIII
+IIZI → +IIIZ
+IIIZ → +IIZI
+XIII → +IXII
+IXII → +XIII
+IIXI → +IIIX
+IIIX → +IIXI

 +ZIII → +ZIII
+IZII → -IXII
+IIZI → +IIZI
+IIIZ → +IIIX
+XIII → +XIII
+IXII → -IZII
+IIXI → +IIXI
+IIIX → +IIIZ

 +ZIII → -IXII
+IZII → +ZIII
+IIZI → +IIIX
+IIIZ → +IIZI
+XIII → -IZII
+IXII → +XIII
+IIXI → +IIIZ
+IIIX → +IIXI

 +ZIII → -XIII
+IZII → +IZII
+IIZI → +IIXI
+IIIZ → +IIIZ
+XIII → -ZIII
+IXII → +IXII
+IIXI → +IIZI
+IIIX → +IIIX

 +ZIII → +IZII
+IZII → -XIII
+IIZI → +IIIZ
+IIIZ → +IIXI
+XIII → +IXII
+IXII → -ZIII
+IIXI → +IIIX
+IIIX → +IIZI

 +ZIII → -XIII
+IZII → -IXII
+IIZI → +IIXI
+IIIZ → +IIIX
+XIII → -ZIII
+IXII → -IZII
+IIXI → +IIZI
+IIIX → +IIIZ

 +ZIII → -IXII
+IZII → -XIII
+IIZI → +IIIX
+IIIZ → +IIXI
+XIII → -IZII
+IXII → -ZIII
+IIXI → +IIIZ
+IIIX → +IIZI

 +ZIII → -ZIII
+IZII → +IZII
+IIZI → -IIZI
+IIIZ 